In [ ]:
## Compute mean spectrum of a Dataset


from scipy import interpolate
from scipy.signal import find_peaks

def peak_picking(files,max_peaks):
    
    ints_n = 0
    
    min_mz = 0
    max_mz = 0 
    
    for f in files:
        ms = np.load(f)
        mzs = ms[0,]
        
        if min_mz > np.min(mzs):
            min_mz = np.min(mzs)
            
        if max_mz < np.max(mzs):
            max_mz = np.max(mzs)    
    
    for f in files:
        ms = np.load(f)

        mzs = ms[0,]
        ints = ms[1,]

        mzs_n = np.arange(min_mz,max_mz, 0.0001)
        
        f = interpolate.interp1d(mzs , ints,fill_value="extrapolate")
        ints_n = ints_n + f(mzs_n)
    
    peaks, _ = find_peaks(ints_n,distance=100)
    instensity_mean = ints_n[peaks]
    mz_peaks = mzs_n[peaks]

    peaks = peaks[np.argsort(ints_n[peaks])[::-1][:max_peaks]]

    return(np.sort(mzs_n[peaks]))

mass = peak_picking(["MSIa.mean.npy","MSIb.mean.npy"])
np.savetxt("./DS/MSI/MSIa_MSIb.peaks", mass, delimiter=",")

# create datacube
#nohup python create_msi_da.py -i "MSIa.imzML" -i2 "./DS/MSI/MSIa_MSIb.peaks" -tol 0.01 -o1 "./DS/MSIa" &

In [ ]:
# Compute datacube of a dataset

import numpy as np
from numpy import genfromtxt
import pandas as pd


def check_mass_database(experimental_mass, database_mass, tolerance):
    if database_mass != 0:
        return abs((experimental_mass - database_mass)) <= tolerance
    
    
def binarySearch_tol(arr, l, r, x, tolerance): 
  
    while l <= r: 
  
        mid = l + (r - l)//2; 
          
        # Check if x is present at mid 
        if check_mass_database(x,arr[mid],tolerance): 
            # check the mass around 
            itpos = mid +1
            itneg = mid -1
            index = []
            index.append(mid)
            if( itpos < len(arr)):
                while check_mass_database(x,arr[itpos],tolerance) and itpos < len(arr):
                    index.append(itpos)
                    itpos += 1 
            if( itneg > 0): 
                while check_mass_database(x,arr[itneg],tolerance) and itneg > 0:
                    index.append(itneg)
                    itneg -= 1     
            return index 
  
        # If x is greater, ignore left half 
        elif arr[mid] < x: 
            l = mid + 1
  
        # If x is smaller, ignore right half 
        else: 
            r = mid - 1
      
    # If we reach here, then the element 
    # was not present 
    return -1

def align_peaks(peaks_mz,intensities,database_exactmass, tolerance): 
    pixel_new_vec = np.zeros(np.size(database_exactmass,0))
    for i in range(0,np.size(peaks_mz,0)):
        exp_peak = peaks_mz[i]
        # append the maximum mass possible to avoid going over
        vec_ind = binarySearch_tol(np.append(database_exactmass,np.max(database_exactmass)+1), 0, len(database_exactmass)-1, exp_peak,tolerance)
        if vec_ind != -1:
            if len(vec_ind) > 1:
                for j in range(0,np.size(vec_ind,0)):
                    if intensities[i] > pixel_new_vec[vec_ind[j]]:
                        pixel_new_vec[vec_ind[j]] = intensities[i]
                        
            if len(vec_ind) == 1:
                if intensities[i] > pixel_new_vec[vec_ind]:
                    pixel_new_vec[vec_ind] = intensities[i]
                
    return(pixel_new_vec)

def create_datacube(file,peaks_file,df,tolerance):
    
    msi_mz = genfromtxt(peaks_file, delimiter=' ')
    database_exactmass = np.sort(msi_mz)
    
    msi = np.zeros((np.size(database_exactmass),np.shape(df)[0]))

    i = 0 
    for index, row in df.iterrows():
        
        print(str(row['MSI pixel id']),row['train'])
        path = file + row['MSI name'] +"/spec_" + str(row['MSI pixel id']) + ".npy"
        spec = np.load(path)
        
        mode = row['train']
        
        mzs = spec[:,0]
        intensities = spec[:,2]
        
        peaks_mz = mzs
        vec_int = intensities
        pixel_new_vec = align_peaks(peaks_mz,vec_int,database_exactmass, tolerance)
        msi[:,i] = pixel_new_vec
        i += 1
    
    
    return(msi)


peaks_file = "./DS/MSI/MSIa_MSIb.peaks


tolerance = 0.001
df = pd.read_csv("./DS/Annot_table.csv",sep=",", header=0)

f = "./DS/MSI/centroid_data/param_BGFG1/"

msi = create_datacube(f,peaks_file,df,tolerance)
np.save(f+"msi",msi)

In [3]:
from sklearn.metrics import balanced_accuracy_score
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import balanced_accuracy_score


def rf_classifier(X, y, random_state=0):
    from sklearn.ensemble import RandomForestClassifier
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]


    model = RandomForestClassifier()
    model.fit(X, y)
        
    return model
    
def svm_classifier(X, y, random_state=0):
    from sklearn.svm import SVC
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]


    model = SVC(kernel = "rbf",random_state=random_state)
    model.fit(X, y)

    return model
    
    
def lda_classifier(X, y, random_state=0):
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]

    model = LinearDiscriminantAnalysis()
    model.fit(X, y)
    
    return model

def xgboost_classifier(X, y, random_state=0):
    import xgboost
    
    np.random.seed(random_state)
    arr = np.arange(0,np.shape(y)[0],1)
    np.random.shuffle(arr)
    
    y = y[arr]
    X = X[arr,:]

    model = xgboost.XGBClassifier(n_jobs=8)
    model.fit(X, y)
    
    return model

def normalize(msi):
    
    normalised_msi = msi.copy()
    
    for i in range(0,np.shape(normalised_msi)[1]):
        normalised_msi[:,i] = msi[:,i]/np.sum(msi[:,i])
        
    normalised_msi[np.isnan(normalised_msi)] = 0
        
    return normalised_msi

In [ ]:

df_annot = pd.read_csv("./DS/Annot_table.csv",sep=",", header=0)
results_rf = []
results_lda = []
results_svm = []
results_xgboost = []

f = "./DS/MSI/centroid_data/param_BGFG1/msi.npy"
        
msi = np.load(f )
msi = normalize(np.log(msi+1))
     
msi = msi.astype("float32")
        
for random_state in np.arange(0,15):
    
    model = rf_classifier(np.transpose(msi[:,df_annot["train"].to_numpy() == 1]), df_annot.loc[df_annot["train"] == 1,"Annotations"].to_numpy(),random_state)
            
    y_pred_test = model.predict(np.transpose(msi[:,df_annot["train"].to_numpy() == 0]))
    balanced_accuracy = balanced_accuracy_score(df_annot.loc[df_annot["train"] == 0,"Annotations"],y_pred_test)
        
    print(balanced_accuracy)

    results_rf.append(balanced_accuracy)
            
for random_state in np.arange(0,3):

    model = lda_classifier(np.transpose(msi[:,df_annot["train"].to_numpy() == 1]), df_annot.loc[df_annot["train"] == 1,"Annotations"].to_numpy(),random_state)
            
    y_pred_test = model.predict(np.transpose(msi[:,df_annot["train"].to_numpy() == 0]))
    balanced_accuracy = balanced_accuracy_score(df_annot.loc[df_annot["train"] == 0,"Annotations"],y_pred_test)
 
    print(balanced_accuracy)

    results_lda.append(balanced_accuracy)
                                   
for random_state in np.arange(0,3):
    
    model = svm_classifier(np.transpose(msi[:,df_annot["train"].to_numpy() == 1]), df_annot.loc[df_annot["train"] == 1,"Annotations"].to_numpy(),random_state)
            
    y_pred_test = model.predict(np.transpose(msi[:,df_annot["train"].to_numpy() == 0]))
    balanced_accuracy = balanced_accuracy_score(df_annot.loc[df_annot["train"] == 0,"Annotations"],y_pred_test)

    print(balanced_accuracy)

    results_svm.append(balanced_accuracy)

                                   
for random_state in np.arange(0,3):
    
    model = xgboost_classifier(np.transpose(msi[:,df_annot["train"].to_numpy() == 1]), df_annot.loc[df_annot["train"] == 1,"Annotations"].to_numpy(),random_state)
            
    y_pred_test = model.predict(np.transpose(msi[:,df_annot["train"].to_numpy() == 0]))
    balanced_accuracy = balanced_accuracy_score(df_annot.loc[df_annot["train"] == 0,"Annotations"],y_pred_test)

        
    print(balanced_accuracy)

    results_xgboost.append(balanced_accuracy)
        
            
np.savetxt("./DS/results_rf_SpaceMDS_NOsigdeg.csv", results_rf , delimiter=',')
np.savetxt("./DS/results_lda_SpaceMDS_NOsigdeg.csv", results_lda, delimiter=',')
np.savetxt("./DS/results_svm_SpaceMDS_NOsigdeg.csv", results_svm, delimiter=',')
np.savetxt("./DS/results_xgboost_SpaceMDS_NOsigdeg.csv", results_xgboost, delimiter=',')